# Test Python Client

In [1]:
from src.ga_api_package.ga_api_client import System, Repository, HyperLogLog, pprint
import logging
import pandas as pd
logging.basicConfig(
    format='▸ %(asctime)s %(levelname)s %(filename)s:%(funcName)s(%(lineno)d) - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    # level=logging.INFO
)

## System

In [ ]:
system = System('https://rdlab-214.genie-analytics.com:8443/api', 'api', 'api123!@#')

In [ ]:
system_desc = await system.get_system()
pprint(system_desc)

In [ ]:
system_conf = await system.set_system(system_desc['config'])
pprint(system_conf)

In [ ]:
pprint(await system.get_all_tenants())

In [ ]:
tenant_desc = await system.create_tenant(0, {
    "limit":10000000000
})
pprint(tenant_desc)

In [ ]:
tenant_conf = await system.update_tenant(0, tenant_desc['config'])
pprint(tenant_conf)

In [ ]:
tenant_conf = await system.get_tenant(0)
pprint(tenant_conf)

In [ ]:
# await system.delete_tenant(0)

## Repository

In [2]:
repo = Repository('https://rdlab-214.genie-analytics.com/api', 'api', 'default', 'api123!@#')

### Using Dataset

In [ ]:
# delete all datasets to have a clean environment
# await repo.del_all_datasets()

In [ ]:
dset_id = 13579
dset_conf = {
    'name':'example2',
    'series':'hour',
    'freq':300,
    'retainDepth':86400*7,
    'retainSize':10000000,
    'run':False
}
# create dataset
dset = await repo.set_dataset(dset_id, dset_conf)
# update dataset
dset = await repo.set_dataset(dset_id, dset_conf)
# get dataset
dset = await repo.get_dataset(dset_id)

In [ ]:
pipe_conf = {
    'name':'example',
    'pipeline': {
        'scope': {
            'i-field': '@controller.home',
            'i-entry': [
                [True,'t:0:0:1',0]
            ],
            'mode': '%directional'
        },
        'metric': [
            ['$sum', {
                'field':'@flow.bytes'
            }]
        ]
    }
}
# create pipeline
pipe = await dset.set_pipeline(1001, pipe_conf)
# update pipeline
pipe = await dset.set_pipeline(1001, pipe_conf)
# get pipeline
pipe = await dset.get_pipeline(1001)

In [ ]:
# patch dataset data
dts = pd.date_range('2022-03-11 12:00:00+08:00', end='2022-03-11 13:00:00+08:00', freq='5T')
print(await dset.patch_data(dts))

In [ ]:
# poll dataset data
print(await dset.poll_data(dts))

In [ ]:
# read dataset data
df = await pipe.read_data(dts, columns=['ts','forward','opposite'])
print(df)

In [ ]:
# delete pipeline
await dset.del_pipeline(1001)

In [ ]:
# delete all pipelines
await dset.del_all_pipelines()

In [ ]:
# delete dataset
await repo.del_dataset(dset_id)

In [ ]:
# delete all datasets
# await repo.del_all_datasets()

### HyperLogLog

In [3]:
adhoc = await repo.set_adhoc(300, {
    'scope': {
        'i-field': '@controller.home',
        'i-entry': [
            [True,'t:0:0:1',0]
        ],
        'mode': '%directional'
    },
    'metric': [
        ['$distinct', {
            'fields':['@flow.addr.dst']
        }]
    ]
})
dts = pd.date_range('2022-03-11 12:00:00+08:00', end='2022-03-11 13:00:00+08:00', freq='5T')
df = await adhoc.read_data(dts, 'hour', columns=['ts','forward','opposite'])
print(df)

                          ts forward opposite
0  2022-03-11 12:00:00+08:00   h:245    h:214
1  2022-03-11 12:05:00+08:00   h:213    h:169
2  2022-03-11 12:10:00+08:00   h:218    h:180
3  2022-03-11 12:15:00+08:00   h:156    h:163
4  2022-03-11 12:20:00+08:00   h:151    h:130
5  2022-03-11 12:25:00+08:00   h:172    h:127
6  2022-03-11 12:30:00+08:00   h:153    h:132
7  2022-03-11 12:35:00+08:00   h:196    h:191
8  2022-03-11 12:40:00+08:00   h:163    h:138
9  2022-03-11 12:45:00+08:00   h:166    h:136
10 2022-03-11 12:50:00+08:00   h:183    h:179
11 2022-03-11 12:55:00+08:00   h:145    h:162
12 2022-03-11 13:00:00+08:00   h:182    h:184


In [4]:
df2 = df.set_index('ts').sum()
df2

forward     h:1056
opposite    h:1114
dtype: object

In [5]:
df2.apply(HyperLogLog.value)

forward     1056
opposite    1114
dtype: int64

In [ ]:
await repo.close()